In [121]:
import numpy as np
import json
"""To use: Make text file with shot data including time of impact and location of each pin, and the projectile velocity
in JSON format in shot_data folder (use -0.xx instea of -.xx). Update the code with your file path to shot data. Then run all cells"""
#possibly try using pandas DataFrame instead? or np.genfromtxt. Both need to specify dtype
#Calculate tilt
#Assuming 4 pins
#Pin locations (mm) and time of impact (us):

datafile = r"Z:\shot_data\19-615.txt"

with open(datafile) as f:
    data = json.load(f)
    
#data is a dictionary. Can get individal values using data['vproj'], or data['pin1'][1] for data in list.
#locals function unpacks dictionary keys into local variables.
print(data)
locals().update(data)

pin1.append(t1)
pin2.append(t2)
pin3.append(t3)
pin4.append(t4)
#pins has the format [x,y,z,t]

{'vproj': 1.9799, 'pin1': [-2.81, -9.472, 0], 'pin2': [-6.204, 6.69, 0], 'pin3': [9.486, 1.91, 0], 'pin4': [0, 0, 0], 't1': 18.321, 't2': 18.32724, 't3': 18.33564, 't4': 18.61148}


In [122]:
#define projectile plane with z' vectors (pinX[4])
#pins [x,y,z,t,tz]
pin1.append(t1*vproj)
pin2.append(t2*vproj)
pin3.append(t3*vproj)
pin4.append(t4*vproj)

#choose three pins (a,b,c = 1,2,3, or 4) to use to define the plane, d is the remaining pin
def plane(a,b,c, d):
    """define a plane with 3 points by making 1 and 2 xyz vectors"""
    #define first set of vectors
    xv1 = a[0]-b[0]
    yv1 = a[1]-b[1]
    zv1 = a[4]-b[4]
    
    #second set
    xv2 = a[0]-c[0]
    yv2 = a[1]-c[1]
    zv2 = a[4]-c[4]
    
    #surfaces
    xx = yv1*zv2 - zv1*yv2
    xy = zv1*xv2 - xv1*zv2
    xz = xv1*yv2 - yv1*xv2
    
    #normal vector
    norm = xz/np.sqrt(xx**2 + xy**2 + xz **2)
    #norm is .99999
    #if norm is negative, arccos(norm) = 180 instead of zero (180 degrees = 3141.59 mrad)

    #line of contact angle (in degrees, clockwise from vertical, looking from back of the target towards the projectile)
    phi = 360-np.degrees(np.arctan2(xy,xx))
    
    #tilt in radians
    theta = np.abs(np.arccos(norm))
    theta2 = np.arccos(np.abs(norm)) # I believe theta 2 is the more general case so use that.
    
    #tilt in mrad
    theta_mrad = theta/10**-3
    theta2_mrad = theta2/10**-3
    
    #velocity of line of contact in m/s
    vc = (vproj/np.tan(theta2)) * 10**3
    
    #predicted impact time at final pin (d)
    tfinal = (-xx*(d[0]-a[0]) - xy*(d[1]-a[1])+xz*a[4])/xz/vproj
    
    return theta2_mrad, phi, vc, tfinal, theta_mrad


In [123]:
opt4 = plane(pin2,pin3,pin4, pin1) #center pin is pin1
print(opt4)

(120.24510563628282, 466.46267131701376, 16386.10067595732, 19.21443118900325, 3021.3475479535105)


In [124]:
opt1 = plane(pin1,pin2,pin3, pin4) #center pin is pin4
print(opt1)

(1.737736450853183, 322.6407226790445, 1139354.593216777, 18.32800504652078, 3139.85491713894)


In [125]:
opt2 = plane(pin1,pin2,pin4, pin3)
print(opt2)

(120.48448330793936, 347.7862527643223, 16353.228633813107, 19.20312856179558, 3021.1081702818537)


In [126]:
opt3 = plane(pin1,pin3,pin4, pin2)
print(opt3)

(110.70482976589439, 408.1023036813608, 17811.373969816468, 19.12366235039858, 110.70482976589439)


In [7]:
#uncertainty in tilt
mean_tilt = sum([opt1[0],opt2[0], opt3[0], opt4[0]])/4
stdev_tilt = np.sqrt(sum([(opt1[0]-mean_tilt)**2,(opt2[0]-mean_tilt)**2, (opt3[0]-mean_tilt)**2, (opt4[0]-mean_tilt)**2])/4)
stderr_tilt = stdev_tilt/np.sqrt(4)
print(mean_tilt, stdev_tilt, stderr_tilt)

843.6748084632467 1231.8691192833205 615.9345596416603


In [8]:
#uncertainty in phi
mean_phi = sum([opt1[1],opt2[1], opt3[1], opt4[1]])/4
stdev_phi = np.sqrt(sum([(opt1[1]-mean_phi)**2,(opt2[1]-mean_phi)**2, (opt3[1]-mean_phi)**2, (opt4[1]-mean_phi)**2])/4)
stderr_phi = stdev_phi/np.sqrt(4)
print(mean_phi, stdev_phi, stderr_phi)

0.05639927917957582 0.7537672769528122 0.3768836384764061


In [54]:
#uncertainty in vc
mean_v = sum([opt1[2],opt2[2], opt3[2], opt4[2]])/4
stdev_v = np.sqrt(sum([(opt1[2]-mean_v)**2,(opt2[2]-mean_v)**2, (opt3[2]-mean_v)**2, (opt4[2]-mean_v)**2])/4)
stderr_v = stdev_v/np.sqrt(4)
print(mean_v, stdev_v, stderr_v)

NameError: name 'opt1' is not defined

In [7]:
#Monte Carlo Uncertainty Analysis
import itertools
import time
import matplotlib.pyplot as plt

xy_uncert = .01
t_uncert = .001
arrival_times = []


def monte():
    for _ in itertools.repeat(None, 10000): # repeat N times, save randomly varied new pin, calculate tilt, save to list
        pin2_ = [np.random.normal(pin2[0], xy_uncert), np.random.normal(pin2[1], xy_uncert), np.random.normal(pin2[2], xy_uncert), np.random.normal(pin2[3], t_uncert)] #(mu, sigma, # of points)
        pin2_.append(pin2_[3]*vproj)
        
        pin3_ = [np.random.normal(pin3[0], xy_uncert), np.random.normal(pin3[1], xy_uncert), np.random.normal(pin3[2], xy_uncert), np.random.normal(pin3[3], t_uncert)]
        pin3_.append(pin3_[3]*vproj)
        
        pin4_ = [np.random.normal(pin4[0], xy_uncert), np.random.normal(pin4[1], xy_uncert), np.random.normal(pin4[2], xy_uncert), np.random.normal(pin4[3], t_uncert)]
        pin4_.append(pin4_[3]*vproj)
        
        pin1_ = [np.random.normal(pin1[0], xy_uncert), np.random.normal(pin1[1], xy_uncert), np.random.normal(pin1[2], xy_uncert), np.random.normal(pin1[3], t_uncert)]
        pin1_.append(pin1_[3]*vproj)
        
        temp = plane(pin2_,pin3_,pin4_, pin1_)
        arrival_times.append(temp[3]) #save arrival time at pin 1 to list
        
    
start = time.perf_counter()        
monte()
stop=time.perf_counter()
print("Mean arrival time at center pin: " + str(format(np.mean(arrival_times), '.4f')) + "\nStandard deviation: " + str(format(np.std(arrival_times), '.4f')))
print("Calculation took " + str(format(stop-start, '.2f')) + " seconds.")

plt.hist(arrival_times, bins=100)
plt.show()

Mean arrival time at center pin: 3.7687
Standard deviation: 0.0006
Calculation took 0.55 seconds.


In [27]:
    
    """
    #Find LOC velocity: define a target and projectile plane and find the distance from the line of intersection (line of contact) to a pin
    i = [b[0]-a[0],b[1]-a[1],b[2]-a[2]]
    j = [c[0]-a[0],c[1]-a[1],c[2]-a[2]]
    targetnorm = np.cross(i,j) #array of three numbers xyz
    targetnorm = targetnorm.tolist() #list of xyz
    t = targetnorm + [-targetnorm[0]*a[0]-targetnorm[1]*a[1]-targetnorm[2]*a[2]] # ax+by+cz+d=0,where d=-(ax0+by0+cz0)
    
    #plane of height of projectile above target at time 0
    k = [b[0]-a[0],b[1]-a[1],b[4]-a[4]]
    l = [c[0]-a[0],c[1]-a[1],c[4]-a[4]]
    projectnorm = np.cross(k,l) #array of three numbers xyz
    projectnorm = projectnorm.tolist()
    p = projectnorm + [-projectnorm[0]*a[0]-projectnorm[1]*a[1]-projectnorm[2]*a[4]] # ax+by+cz+d=0,where d=-(ax0+by0+cz0)
    
    dvector = np.cross(targetnorm, projectnorm).tolist()
    #find a point on both planes. Fix z=0, solve norm equations for x and y. Get line of intersection vector.
    if t[0] >= -.00001 and t[0] <= .00001:
        zprime = -t[3]/t[2]
        yprime = ((p[2]*t[3]/t[2])-p[3])/p[1]
        xprime = 0
        LOIvector = [xprime,yprime,zprime]
        
    else:    
        x = ((p[1]*t[3]/t[1])-p[3])/(p[0]-(p[1]*t[0]/t[1]))
        y = (-t[3]-t[0]*x)/t[1]
        z = 0
        LOIvector = [x,y,z]
    
    #find shortest distance from line of intersection to the furthest away pin on target plane (whichever has max travel time)
    #np.inner(a,b) returns a scalar
    times = [a[3],b[3],c[3]]
    longesttime = max(times)
    if times.index(longesttime) == 0:
        point = [a[0],a[1],a[2]]
    elif times.index(longesttime) == 1:
        point = [b[0],b[1],b[2]]
    elif times.index(longesttime) == 2:
        point = [c[0],c[1],c[2]]
    
    dist = np.subtract(LOIvector, point)
    dsquared = ((np.inner(dist,dist)*np.inner(dvector,dvector)) - (np.inner(dist,dvector))**2)/(np.inner(dvector,dvector))
    
    d = np.sqrt(dsquared)
    velocity = d/longesttime #velocity of line of contact in mm/us
    velocity_ms = velocity*10**3
    """

'\n#Find LOC velocity: define a target and projectile plane and find the distance from the line of intersection (line of contact) to a pin\ni = [b[0]-a[0],b[1]-a[1],b[2]-a[2]]\nj = [c[0]-a[0],c[1]-a[1],c[2]-a[2]]\ntargetnorm = np.cross(i,j) #array of three numbers xyz\ntargetnorm = targetnorm.tolist() #list of xyz\nt = targetnorm + [-targetnorm[0]*a[0]-targetnorm[1]*a[1]-targetnorm[2]*a[2]] # ax+by+cz+d=0,where d=-(ax0+by0+cz0)\n\n#plane of height of projectile above target at time 0\nk = [b[0]-a[0],b[1]-a[1],b[4]-a[4]]\nl = [c[0]-a[0],c[1]-a[1],c[4]-a[4]]\nprojectnorm = np.cross(k,l) #array of three numbers xyz\nprojectnorm = projectnorm.tolist()\np = projectnorm + [-projectnorm[0]*a[0]-projectnorm[1]*a[1]-projectnorm[2]*a[4]] # ax+by+cz+d=0,where d=-(ax0+by0+cz0)\n\ndvector = np.cross(targetnorm, projectnorm).tolist()\n#find a point on both planes. Fix z=0, solve norm equations for x and y. Get line of intersection vector.\nif t[0] >= -.00001 and t[0] <= .00001:\n    zprime = -t[3]/t